# `Summarize text in machine Learning`

### `Importing necessaries package` 

In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

In [2]:
# importing the data

df = pd.read_csv('https://raw.githubusercontent.com/amankharwal/Website-data/master/tennis.csv')

In [3]:
df

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,Federer won the Swiss Indoors last week by bea...,https://www.express.co.uk/sport/tennis/1038186...


In [4]:
df.shape

(8, 3)

In [5]:
df['article_text'][0]

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same 

In [6]:
# split the sequences into the data by tokenizing them using sent tokenize

from nltk.tokenize import sent_tokenize
sentences = []
for sentence in df['article_text']:
    sentences.append(sent_tokenize(sentence))

In [7]:
sentences = [y for x in sentences for y in x]

In [8]:
sentences[2]

'I think everyone knows this is my job here.'

In [9]:
word_embeddings = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype = 'float32')
    word_embeddings[word] = coefs
f.close()

In [10]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

In [11]:
clean_sentences = [s.lower() for s in clean_sentences]

In [12]:
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [13]:
clean_sentences

['maria sharapova basically friends tennis players wta tour.',
 "russian player problems openly speaking recent interview said: 'i really hide feelings much.",
 'think everyone knows job here.',
 "i'm courts i'm court playing, i'm competitor want beat every single person whether they're locker room across net.so i'm one strike conversation weather know next minutes go try win tennis match.",
 "i'm pretty competitive girl.",
 "say hellos, i'm sending players flowers well.",
 "uhm, i'm really friendly close many players.",
 "lot friends away courts.'",
 'said really close lot players, something strategic doing?',
 "different men's tour women's tour?",
 "'no, all.",
 'think sport mean friends everyone categorized, tennis player, going get along tennis players.',
 'think every person different interests.',
 "friends completely different jobs interests, i've met different parts life.",
 "think everyone thinks we're tennis players greatest friends.",
 'ultimately tennis small part do.',
 "ma

In [14]:
# Create vector for sentences
sentence_vector = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w,np.zeros((100,))) for w in i.split()]) / (len(i.split()) + 0.001)
    else:
        v = np.zeros((100,))
    sentence_vector.append(v)

In [15]:
# Finding Similarities to Summarize Text

sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vector[i].reshape(1,100), sentence_vector[j].reshape(1,100))[0,0]

In [21]:
import networkx as nx
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [22]:
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
for i in range(5):
    print("ARTICLE")
    print(df['article_text'][i])
    print('\n')
    print("SUMMARY:")
    print(ranked_sentences[i][1])
    print('\n')

ARTICLE
Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in th

In [20]:
scores = nx.pagerank(nx_graph)  # Call the pagerank function
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
for i in range(5):
    print("ARTICLE")
    print(df['article_text'][i])
    print('\n')
    print("SUMMARY:")
    print(ranked_sentences[i][1])
    print('\n')


ARTICLE
Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in th

In [ ]:
# Assuming you have defined `sentences` and `scores` somewhere in your code

# Combine scores with sentences and sort by scores
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

# Print top 5 sentences along with the original article text
for i in range(5):
    print("ARTICLE")
    print(df['article_text'][i])
    print('\n')
    print("SUMMARY:")
    print(ranked_sentences[i][1])
    print('\n')


In [ ]:
print(type(scores))